In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-09 14:17:30--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

100%[======================================>] 3,832       --.-K/s   in 0.01s   

2024-07-09 14:17:31 (348 KB/s) - ‘minsearch.py’ saved [3832/3832]



In [7]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import os

In [7]:
login(token = os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/wex570/.cache/huggingface/token
Login successful


In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.



===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//vault.rc.hms.harvard.edu'), PosixPath('8200')}
The following directories listed in your path were found to be non-existent: {PosixPath('*4=\\E[3;2~'), PosixPath('AL=\\E[%dL'), PosixPath('Co#8'), PosixPath('kN=\\E[6~'), PosixPath('sr=\\EM'), PosixPath('im=\\E[4h'), PosixPath('us=\\E'), PosixPath('ms'), PosixPath('k4=\\EOS'), PosixPath('kF=\\E[1;2B'), PosixPath('kh=\\E[1~'), PosixPath('F3=\\E[1;2P'), PosixPath('cl=\\E[H\\E[J'), PosixPath('kd=\\EOB'), PosixPath('*7=\\E[1;2F'), PosixPath('ct=\\E[3g'), PosixPath('rc=\\E8'), PosixPath('am'), PosixPath('Km=\\E[M'), PosixPath('F2=\\E[24~'), PosixPath('co#120'), PosixPath('k2=\\EOQ'), PosixPath('F8=\\E[17;2~'), PosixPath('so=\\E'), PosixPath('ce=\\E[K'), PosixPath('cr=^M'), PosixPath('li#30'), PosixPath('#2=\\E[1;2H'), PosixPath('al=\\E[L'), PosixPath('\\

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [9]:
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")


NameError: name 'tokenizer' is not defined

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    messages = [ 
        {"role": "system", "content": prompt}
    ] 
    
    
    
    generation_args = { 
        "max_new_tokens": 500, 
        "return_full_text": False, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 
    
    output = pipe(messages, **generation_args) 
    result = output[0]['generated_text']
    return result

In [14]:
rag("I just discovered the course. Can I still join it?")

/home/wex570/anaconda3/envs/inference/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


" Based on the provided context, you can still join the course even if you discover it after the start date. You are eligible to submit homeworks, but there will be deadlines for final projects, so it's recommended not to delay your submissions. The course materials will be available for you to follow at your own pace after the course finishes, and you can continue working on your final capstone project."